In [ ]:
import os

domo_username = os.environ.get("DOMO_USERNAME")
domo_instance = "domo-community"
domo_password = os.environ.get("DOJO_PASSWORD")

In [ ]:
import requests


def get_full_auth(
    domo_instance, domo_username, domo_password
) -> str:  # returns a session token
    """use username and password to generate an access token"""

    url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

    body = {
        "method": "password",
        "emailAddress": domo_username,
        "password": domo_password,
    }

    res = requests.request(method="POST", url=url, json=body)
    data = res.json()

    token = data.get("sessionToken")
    if not token:
        raise Exception("unable to retrieve a session token")

    return token


session_token = get_full_auth(domo_instance, domo_username, domo_password)
assert session_token

In [ ]:
def generate_query_dataflows_body(count=10, offset=0):
    return {
        "entities": ["DATAFLOW"],
        "count": count,
        "offset": offset,
        "combineResults": False,
        "query": "*",
        "filters": [
            {
                "filterType": "term",
                "field": "data_flow_type",
                "value": "MAGIC",
                "name": "Magic ETL v2",
                "not": False,
            }
        ],
    }


body_dataflows = generate_query_dataflows_body(10, 0)
body_dataflows

{'entities': ['DATAFLOW'],
 'count': 10,
 'offset': 0,
 'combineResults': False,
 'query': '*',
 'filters': [{'filterType': 'term',
   'field': 'data_flow_type',
   'value': 'MAGIC',
   'name': 'Magic ETL v2',
   'not': False}]}

In [ ]:
def query_datacenter(
    domo_instance,
    session_token,
    body,
    return_raw: bool = False,
    debug_api: bool = False,
):
    """function that queries the datacenter and for a specific body"""

    url = f"https://{domo_instance}.domo.com/api/search/v1/query"

    if debug_api:
        print({"url": url, "body": body})

    headers = {"x-domo-authentication": session_token}

    res = requests.request(url=url, method="POST", headers=headers, json=body)

    if return_raw:
        return res

    data = res.json()

    return data.get("searchObjects")


dataflows_ls = query_datacenter(
    domo_instance=domo_instance,
    session_token=session_token,
    body=body_dataflows,
    debug_api=True,
    return_raw=False,
)

print(len(dataflows_ls))
dataflows_ls[0]

{'url': 'https://domo-community.domo.com/api/search/v1/query', 'body': {'entities': ['DATAFLOW'], 'count': 10, 'offset': 0, 'combineResults': False, 'query': '*', 'filters': [{'filterType': 'term', 'field': 'data_flow_type', 'value': 'MAGIC', 'name': 'Magic ETL v2', 'not': False}]}}
10


{'entityType': 'dataflow',
 'databaseId': '131',
 'searchId': {'indexName': None,
  'databaseId': '131',
  'customer': 'mmmm-0012-0200',
  'entityType': 'dataflow'},
 'createDate': 1678147044000,
 'lastModified': 1678147835000,
 'lastIndexed': 1680574033100,
 'highlightedFields': {},
 'language': 'English',
 'requestAccess': False,
 'score': 10.000001,
 'name': 'BeastModesGovernance_01_AccumulateHistory',
 'description': '',
 'ownedById': '1893952720',
 'ownedByName': 'Jae Wilson1',
 'tags': [],
 'lastRunDate': 1678147669000,
 'inputDatasets': [{'name': 'Governance_Beast Modes',
   'id': 'b7998b45-ccb5-4183-830e-8c2bdd489fd0'}],
 'outputDatasets': [{'name': 'Governance_BeastMode_historical',
   'id': 'e6559952-30ce-41a1-b831-f29ebe95d6e5'}],
 'runCount': 2,
 'successRate': 1.0,
 'dataFlowType': 'MAGIC',
 'status': 'SUCCESS',
 'deleted': False,
 'passwordProtected': False,
 'abandoned': False,
 'owners': [{'id': '1893952720',
   'type': 'USER',
   'displayName': 'Jae Wilson1'}],
 'owner

In [ ]:
dataflow_ids = [dataflow.get("databaseId") for dataflow in dataflows_ls]
dataflow_ids

['131', '227', '37', '185', '38', '226', '124', '46', '81', '75']

In [ ]:
def get_dataflow_definition(dataflow_id, session_token, return_raw: bool = False):
    url = f"https://{domo_instance}.domo.com/api/dataprocessing/v1/dataflows/{dataflow_id}"

    headers = {"x-domo-authentication": session_token}

    res = requests.request(method="get", url=url, headers=headers)

    if return_raw:
        return res

    data = res.json()

    return data


get_dataflow_definition(dataflow_ids[0], session_token=session_token)

{'id': 131,
 'name': 'BeastModesGovernance_01_AccumulateHistory',
 'dapDataFlowId': '564d37be-bc96-485f-a854-ef3985864551',
 'responsibleUserId': 1893952720,
 'runState': 'ENABLED',
 'onboardFlowVersion': {'id': 520,
  'timeStamp': 1678147835000,
  'authorId': 1893952720,
  'numInputs': 1,
  'numOutputs': 1,
  'executionCount': 0,
  'executionSuccessCount': 0,
  'versionNumber': 3},
 'lastExecution': {'id': 272919,
  'onboardFlowId': 131,
  'previewRows': 0,
  'dapDataFlowExecutionId': '4759ab00-3f9d-4d55-9087-32aa38655a51',
  'beginTime': 1678147659000,
  'endTime': 1678147669000,
  'lastUpdated': 1678147669000,
  'failed': False,
  'state': 'SUCCESS',
  'dataFlowVersion': 519},
 'created': 1678147044000,
 'modified': 1678147835000,
 'actions': [{'type': 'LoadFromVault',
   'id': '40554fc4-428a-4117-8820-b3f4f4927021',
   'name': 'Governance_Beast Modes',
   'gui': {'x': 312,
    'y': 132,
    'color': 3238043,
    'colorSource': None,
    'sampleJson': None},
   'dataSourceId': 'b799

In [ ]:
dataflow_definition_ls = [
    get_dataflow_definition(dataflow_id, session_token=session_token)
    for dataflow_id in dataflow_ids
]
dataflow_definition_ls[0]

{'id': 131,
 'name': 'BeastModesGovernance_01_AccumulateHistory',
 'dapDataFlowId': '564d37be-bc96-485f-a854-ef3985864551',
 'responsibleUserId': 1893952720,
 'runState': 'ENABLED',
 'onboardFlowVersion': {'id': 520,
  'timeStamp': 1678147835000,
  'authorId': 1893952720,
  'numInputs': 1,
  'numOutputs': 1,
  'executionCount': 0,
  'executionSuccessCount': 0,
  'versionNumber': 3},
 'lastExecution': {'id': 272919,
  'onboardFlowId': 131,
  'previewRows': 0,
  'dapDataFlowExecutionId': '4759ab00-3f9d-4d55-9087-32aa38655a51',
  'beginTime': 1678147659000,
  'endTime': 1678147669000,
  'lastUpdated': 1678147669000,
  'failed': False,
  'state': 'SUCCESS',
  'dataFlowVersion': 519},
 'created': 1678147044000,
 'modified': 1678147835000,
 'actions': [{'type': 'LoadFromVault',
   'id': '40554fc4-428a-4117-8820-b3f4f4927021',
   'name': 'Governance_Beast Modes',
   'gui': {'x': 312,
    'y': 132,
    'color': 3238043,
    'colorSource': None,
    'sampleJson': None},
   'dataSourceId': 'b799

In [ ]:
def get_action_list(dd_obj):
    return [
        {
            "dataflow_id": dd_obj.get("id"),
            "datafow_name": dd_obj.get("name"),
            "action_type": action.get("type"),
        }
        for action in dd_obj.get("actions")
    ]


get_action_list(dd_obj=dataflow_definition_ls[0])

[{'dataflow_id': 131,
  'datafow_name': 'BeastModesGovernance_01_AccumulateHistory',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 131,
  'datafow_name': 'BeastModesGovernance_01_AccumulateHistory',
  'action_type': 'ExpressionEvaluator'},
 {'dataflow_id': 131,
  'datafow_name': 'BeastModesGovernance_01_AccumulateHistory',
  'action_type': 'PublishToVault'}]

In [ ]:
action_ls = [get_action_list(dd_obj) for dd_obj in dataflow_definition_ls]

flat_action_ls = [action for a_list in action_ls for action in a_list]
flat_action_ls[0:5]

[{'dataflow_id': 131,
  'datafow_name': 'BeastModesGovernance_01_AccumulateHistory',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 131,
  'datafow_name': 'BeastModesGovernance_01_AccumulateHistory',
  'action_type': 'ExpressionEvaluator'},
 {'dataflow_id': 131,
  'datafow_name': 'BeastModesGovernance_01_AccumulateHistory',
  'action_type': 'PublishToVault'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'WindowAction'}]

In [ ]:
import pandas as pd

df = pd.DataFrame(flat_action_ls)
df[0:5]

,dataflow_id,datafow_name,action_type
0,131,BeastModesGovernance_01_AccumulateHistory,LoadFromVault
1,131,BeastModesGovernance_01_AccumulateHistory,ExpressionEvaluator
2,131,BeastModesGovernance_01_AccumulateHistory,PublishToVault
3,227,DDX_Landing Page_QA,LoadFromVault
4,227,DDX_Landing Page_QA,WindowAction


In [ ]:
df.to_csv("marks_csv.csv")